## Optimization Model

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from gurobipy import Model, GRB, quicksum

**Load Data**

In [2]:
df = pd.read_csv('Data/cleaned_data/all_data_OD.csv')

In [3]:
df['transport_demand'] = df['cars_7am'] + df['cars_8am'] +df['cars_4pm'] + df['cars_5pm']
demand_matrix_df = df.pivot(index='origin', columns='dest', values='transport_demand')

In [4]:
demand_matrix_df = demand_matrix_df.fillna(0)
d = demand_matrix_df.values

In [41]:
average_co2 = sum(d[i, j] * TZ_car[i, j] for i in range(Z) for j in range(Z))/sum(d[i, j]for i in range(Z) for j in range(Z))
average_co2

450.5775613083713

In [6]:
M = np.max(d)
M

245.097

__Get emissions and travel times__

In [7]:
# Zones to Zones by Car
zones_to_zones_car1 = pd.read_csv("Data/cleaned_data/navitia_zones_to_zones_car_cleaned.csv")
zones_to_zones_car2 = zones_to_zones_car1.copy()
zones_to_zones_car2[['origin_zone', 'destination_zone']] = zones_to_zones_car1[['destination_zone', 'origin_zone']]
zones_to_zones_car = pd.concat([zones_to_zones_car1, zones_to_zones_car2], ignore_index=True)

C02_zones_to_zones_car = zones_to_zones_car.pivot(index='origin_zone', columns='destination_zone', values='route_CO2').fillna(0)
EZ_car = C02_zones_to_zones_car.values
time_zones_to_zones_car = zones_to_zones_car.pivot(index='origin_zone', columns='destination_zone', values='duration').fillna(0)
TZ_car = time_zones_to_zones_car.values

In [8]:
# Zones to Hubs by Car
zones_to_hubs_car = pd.read_csv("Data/cleaned_data/navitia_zones_to_hubs_car_cleaned.csv")
C02_zones_to_hubs_car = zones_to_hubs_car.pivot(index='origin_zone', columns='destination_hub', values='route_CO2')
EH_car = C02_zones_to_hubs_car.values
time_zones_to_hubs_car = zones_to_hubs_car.pivot(index='origin_zone', columns='destination_hub', values='duration')
TH_car = time_zones_to_hubs_car.values

In [9]:
# Zones to Hubs by Public Transport
zones_to_hubs_public = pd.read_csv("Data/cleaned_data/navitia_zones_to_hubs_public_cleaned.csv")
C02_zones_to_hubs_public = zones_to_hubs_public.pivot(index='origin_zone', columns='destination_hub', values='route_CO2')
EH_public = C02_zones_to_hubs_public.values
time_zones_to_hubs_public = zones_to_hubs_public.pivot(index='origin_zone', columns='destination_hub', values='duration')
TH_public = time_zones_to_hubs_public.values

In [10]:
# Hubs to Zones by Public Transport
hubs_to_zones_public = pd.read_csv("Data/cleaned_data/navitia_hubs_to_zones_public_cleaned.csv")
C02_hubs_to_zones_public = hubs_to_zones_public.pivot(index='origin_hub', columns='destination_zone', values='route_CO2')
EZ_public = C02_hubs_to_zones_public.values
time_hubs_to_zones_public = hubs_to_zones_public.pivot(index='origin_hub', columns='destination_zone', values='duration')
TZ_public = time_hubs_to_zones_public.values

In [11]:
# Zones to Hubs/Hubs to Zones by Bike
zones_to_hubs_bike = pd.read_csv("Data/cleaned_data/navitia_zones_to_hubs_bike_cleaned.csv")
time_zones_to_hubs_bike = zones_to_hubs_bike.pivot(index='zone', columns='hub', values='duration')
TH_bike = time_zones_to_hubs_bike.values
time_hubs_to_zones_bike = zones_to_hubs_bike.pivot(index='hub', columns='zone', values='duration')
TZ_bike = time_hubs_to_zones_bike.values

**Initialize Hubs**

In [12]:
bike_distance = pd.read_csv('Data/cleaned_data/bike_distance_matrix.csv', index_col=0)

# Get list of hubs
hubs = bike_distance.drop_duplicates(subset=['latitude','longitude'])[['label','latitude','longitude']]
hubs = hubs.reset_index()
hubs = hubs.drop(['index','label'], axis=1)
hubs = hubs.reset_index()
hubs.columns = ['hub_id','latitude','longitude']

In [13]:
hubs['existing'] = 1
hubs.loc[0:42, 'existing'] = 0
hubs

,hub_id,latitude,longitude,existing
0,0,50.878500,4.706625,0
1,1,50.872565,4.689152,0
2,2,50.883288,4.706247,0
3,3,50.880325,4.706609,0
4,4,50.881597,4.714705,0
...,...,...,...,...
98,98,50.902958,4.700576,1
99,99,50.871633,4.675939,1
100,100,50.893824,4.689373,1
101,101,50.870495,4.668255,1


**Initialize Parkings**

In [14]:
park = pd.read_csv('Data/cleaned_data/cleaned_parkings.csv', index_col=0)
park = park[['latitude','longitude','number_of_spots']]

parkings = pd.merge(hubs, park, how='left', on=['latitude','longitude'])

# Assumption of parkings per already existing hub
parkings_per_hub = 50

parkings['number_of_spots'].fillna(parkings_per_hub, inplace=True)
P = parkings['number_of_spots'].values # Parking Spots

**Create Connection Matrices:**

In [15]:
# BIKE

# Setting threshold
t_bike = 30 * 60 # Seconds by bike

# CN_BH
CN_BH = time_zones_to_hubs_bike.applymap(lambda x: 1 if x < t_bike else 0).values

#CN_BZ
CN_BZ = time_hubs_to_zones_bike.applymap(lambda x: 1 if x < t_bike else 0).values

/var/folders/6x/002d9wvx0ln2cycd86pjbd740000gn/T/ipykernel_15987/1705460274.py:7: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  CN_BH = time_zones_to_hubs_bike.applymap(lambda x: 1 if x < t_bike else 0).values
/var/folders/6x/002d9wvx0ln2cycd86pjbd740000gn/T/ipykernel_15987/1705460274.py:10: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  CN_BZ = time_hubs_to_zones_bike.applymap(lambda x: 1 if x < t_bike else 0).values


In [16]:
# PUBLIC

# Setting threshold
t_pub = 60 * 60 # seconds by public trans

# CN_PH
CN_PH = time_hubs_to_zones_public.applymap(lambda x: 1 if x < t_pub else 0).values.transpose()

# CN_PZ
CN_PZ = time_zones_to_hubs_public.applymap(lambda x: 1 if x < t_pub else 0).values.transpose()

/var/folders/6x/002d9wvx0ln2cycd86pjbd740000gn/T/ipykernel_15987/1829830833.py:7: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  CN_PH = time_hubs_to_zones_public.applymap(lambda x: 1 if x < t_pub else 0).values.transpose()
/var/folders/6x/002d9wvx0ln2cycd86pjbd740000gn/T/ipykernel_15987/1829830833.py:10: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  CN_PZ = time_zones_to_hubs_public.applymap(lambda x: 1 if x < t_pub else 0).values.transpose()


**Define Parameters:**

In [17]:
beta = 0.7
gamma = 0.86
pi = 0.3

# Hubs
K = len(hubs)
MH = 15

# Number of zones
Z = len(d)

# Bike Capacity
B = 100 * np.ones(K)

**Multi-Objective Model:**

In [18]:
model = Model()
upperbound = M

# Decision variables
a = model.addVars(K, vtype=GRB.BINARY, name="a")
CD = model.addVars(Z, Z, lb=0, ub=M, vtype=GRB.CONTINUOUS, name="CD")
CH = model.addVars(Z, K, Z, lb=0, ub=M, vtype=GRB.CONTINUOUS, name="CH")
PH = model.addVars(Z, K, Z, lb=0, ub=M, vtype=GRB.CONTINUOUS, name="PH")
BH = model.addVars(Z, K, Z, lb=0, ub=M, vtype=GRB.CONTINUOUS, name="BH")
PZ = model.addVars(Z, K, Z, lb=0, ub=M, vtype=GRB.CONTINUOUS, name="PZ")
BZ = model.addVars(Z, K, Z, lb=0, ub=M, vtype=GRB.CONTINUOUS, name="BZ")

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-22


In [19]:
%%time
# Multimodal hubs constraints
model.addConstr(quicksum(a[k] for k in range(K)) <= MH, name="i_c0")

# Big-M constraints for all continuous variables
model.addConstrs((BH[i, k, j] <= M * CN_BH[i, k] * a[k] for i in range(Z) for k in range(K) for j in range(Z)), name="i_c1")
model.addConstrs((PH[i, k, j] <= M * CN_PH[i, k] * a[k] for i in range(Z) for k in range(K) for j in range(Z)), name="i_c2")
model.addConstrs((BZ[i, k, j] <= M * CN_BZ[k, j] * a[k] for i in range(Z) for k in range(K) for j in range(Z)), name="i_c3")
model.addConstrs((PZ[i, k, j] <= M * CN_PZ[k, j] * a[k] for i in range(Z) for k in range(K) for j in range(Z)), name="i_c4")
model.addConstrs((CH[i, k, j] <= M * a[k] for i in range(Z) for k in range(K) for j in range(Z)), name="i_c5")

# Flow constraint
model.addConstrs((CH[i, k, j] + BH[i, k, j] + PH[i, k, j] == BZ[i, k, j] + PZ[i, k, j]
                  for i in range(Z) for k in range(K) for j in range(Z)), name="c1")

# Demand satisfaction
model.addConstrs((quicksum(CH[i, k, j] + BH[i, k, j] + PH[i, k, j] for k in range(K)) + CD[i, j] == d[i, j]
                  for i in range(Z) for j in range(Z)), name="c2")

# Bike capacity
model.addConstrs((quicksum(BZ[i, k, j] for i in range(Z) for j in range(Z)) <= B[k] * beta
                   for k in range(K)), name="c3")

# Parking capacity
model.addConstrs((quicksum(PZ[i, k, j] for i in range(Z) for j in range(Z)) <= P[k] * pi
                  for k in range(K)), name="c4")
print('Constraints defined')

Constraints defined
CPU times: user 10min 13s, sys: 46.8 s, total: 11min
Wall time: 11min 18s


In [20]:
# Objective function
model.setObjective(
                    quicksum(CD[i, j] * EZ_car[i, j] for i in range(Z) for j in range(Z)) + 
                    quicksum(CH[i, k, j] * EH_car[i, k] for i in range(Z) for k in range(K) for j in range(Z)) +
                    quicksum(PH[i, k, j] * EH_public[i, k] for i in range(Z) for k in range(K) for j in range(Z)) +
                    quicksum(PZ[i, k, j] * EZ_public[k, j] for i in range(Z) for k in range(K) for j in range(Z)) +
                    gamma * (
                        quicksum(CD[i, j] * TZ_car[i, j] for i in range(Z) for j in range(Z)) + 
                        quicksum(CH[i, k, j] * TH_car[i, k] for i in range(Z) for k in range(K) for j in range(Z)) +
                        quicksum(BH[i, k, j] * TH_bike[i, k] for i in range(Z) for k in range(K) for j in range(Z)) +
                        quicksum(BZ[i, k, j] * TZ_bike[k, j] for i in range(Z) for k in range(K) for j in range(Z)) +
                        quicksum(PH[i, k, j] * TH_public[i, k] for i in range(Z) for k in range(K) for j in range(Z)) +
                        quicksum(PZ[i, k, j] * TZ_public[k, j] for i in range(Z) for k in range(K) for j in range(Z))
                        ), 
                    GRB.MINIMIZE)

print('Objective defined')

Objective defined


In [38]:
# Optimize the model
model.optimize()

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[arm])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 35952348 rows, 29969899 columns and 114717996 nonzeros
Model fingerprint: 0x6c6f18ab
Variable types: 29969796 continuous, 103 integer (103 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [9e-01, 2e+04]
  Bounds range     [1e+00, 2e+02]
  RHS range        [1e-03, 3e+02]

Loaded MIP start from previous solve with objective 8.52579e+07

Presolve removed 0 rows and 0 columns (presolve time = 6s) ...
Presolve removed 0 rows and 241 columns (presolve time = 13s) ...
Presolve removed 14401294 rows and 241 columns (presolve time = 15s) ...
Presolve removed 14401294 rows and 14130410 columns (presolve time = 20s) ...
Presolve removed 16390102 rows and 14152227 columns (presolve time = 26s) ...
Presolve removed 21738249 rows and 19473754 columns (presolve time = 30s) ...
Presolve 

In [43]:
folder_path = 'results_15_0.86'

In [44]:
hub_solution = np.array([a[i].X for i in range(K)])
if hub_solution is not None:
    np.savetxt(folder_path+'/Hub_solution.txt', hub_solution, fmt='%f')
    print('Hub solution saved to "Hub_solution.txt"')

CD_solution = np.array([[CD[i, j].X for j in range(Z)] for i in range(Z)])
if CD_solution is not None:
    np.save(folder_path+'/CD_solution.npy', CD_solution)
    print('CD solution saved to "CD_solution.npy"')

CH_solution = np.array([[[CH[i, k, j].X for j in range(Z)] for k in range(K)] for i in range(Z)])
if CH_solution is not None:
    np.save(folder_path+'/CH_solution.npy', CH_solution)
    print('CH solution saved to "CH_solution.npy"')

PH_solution = np.array([[[PH[i, k, j].X for j in range(Z)] for k in range(K)] for i in range(Z)])
if PH_solution is not None:
    np.save(folder_path+'/PH_solution.npy', PH_solution)
    print('PH solution saved to "PH_solution.npy"')

BH_solution = np.array([[[BH[i, k, j].X for j in range(Z)] for k in range(K)] for i in range(Z)])
if BH_solution is not None:
    np.save(folder_path+'/BH_solution.npy', BH_solution)
    print('BH solution saved to "BH_solution.npy"')

PZ_solution = np.array([[[PZ[i, k, j].X for j in range(Z)] for k in range(K)] for i in range(Z)])
if PZ_solution is not None:
    np.save(folder_path+'/PZ_solution.npy', PZ_solution)
    print('PZ solution saved to "PZ_solution.npy"')

BZ_solution = np.array([[[BZ[i, k, j].X for j in range(Z)] for k in range(K)] for i in range(Z)])
if BZ_solution is not None:
    np.save(folder_path+'/BZ_solution.npy', BZ_solution)
    print('BZ solution saved to "BZ_solution.npy"')

Hub solution saved to "Hub_solution.txt"
CD solution saved to "CD_solution.npy"
CH solution saved to "CH_solution.npy"
PH solution saved to "PH_solution.npy"
BH solution saved to "BH_solution.npy"
PZ solution saved to "PZ_solution.npy"
BZ solution saved to "BZ_solution.npy"


In [41]:
from utils_aws import process_and_push

number_of_hubs = MH

BH_data = np.load(folder_path+'/BH_solution.npy')
process_and_push(BH_data, type='ikj_to_hub', name_of_file='BH_solution.csv', number_of_hubs=number_of_hubs)
BZ_data = np.load(folder_path+'/BZ_solution.npy')
process_and_push(BZ_data, type='ikj_to_zone', name_of_file='BZ_solution.csv', number_of_hubs=number_of_hubs)
CD_data = np.load(folder_path+'/CD_solution.npy')
process_and_push(CD_data, type='ij', name_of_file='CD_solution.csv', number_of_hubs=number_of_hubs)
CH_data = np.load(folder_path+'/CH_solution.npy')
process_and_push(CH_data, type='ikj_to_hub', name_of_file='CH_solution.csv', number_of_hubs=number_of_hubs)
PH_data = np.load(folder_path+'/PH_solution.npy')
process_and_push(PH_data, type='ikj_to_hub', name_of_file='PH_solution.csv', number_of_hubs=number_of_hubs)
PZ_data = np.load(folder_path+'/PZ_solution.npy')
process_and_push(PZ_data, type='ikj_to_zone', name_of_file='PZ_solution.csv', number_of_hubs=number_of_hubs)
a_hub = np.loadtxt(folder_path+'/Hub_solution.txt', dtype=float)
process_and_push(a_hub, type='hubs', name_of_file='Hub_solution.csv', number_of_hubs=number_of_hubs)

### Result Analysis

In [45]:
a_hub = np.loadtxt(folder_path+'/Hub_solution.txt', dtype=float)
sum(a_hub)

10.0

In [46]:
BH_data = np.load(folder_path+'/BH_solution.npy')
BZ_data = np.load(folder_path+'/BZ_solution.npy')
CD_data = np.load(folder_path+'/CD_solution.npy')
CH_data = np.load(folder_path+'/CH_solution.npy')
PH_data = np.load(folder_path+'/PH_solution.npy')
PZ_data = np.load(folder_path+'/PZ_solution.npy')

In [47]:
#emissions
emissions_opt = sum(CD_data[i, j] * EZ_car[i, j] for i in range(Z) for j in range(Z)) + \
                sum(CH_data[i, k, j] * EH_car[i, k] for i in range(Z) for k in range(K) for j in range(Z)) + \
                sum(PH_data[i, k, j] * EH_public[i, k] for i in range(Z) for k in range(K) for j in range(Z)) + \
                sum(PZ_data[i, k, j] * EZ_public[k, j] for i in range(Z) for k in range(K) for j in range(Z))

In [48]:
time_opt = sum(CD_data[i, j] * TZ_car[i, j] for i in range(Z) for j in range(Z)) + \
           sum(CH_data[i, k, j] * TH_car[i, k] for i in range(Z) for k in range(K) for j in range(Z)) + \
           sum(BH_data[i, k, j] * TH_bike[i, k] for i in range(Z) for k in range(K) for j in range(Z)) + \
           sum(BZ_data[i, k, j] * TZ_bike[k, j] for i in range(Z) for k in range(K) for j in range(Z)) + \
           sum(PH_data[i, k, j] * TH_public[i, k] for i in range(Z) for k in range(K) for j in range(Z)) + \
           sum(PZ_data[i, k, j] * TZ_public[k, j] for i in range(Z) for k in range(K) for j in range(Z))

In [49]:
emissions_opt, time_opt

(60771146.63211083, 28472924.071000002)

In [50]:
baseline_time = 25576671.201000005
baseline_emissions = 65676025.34070091

In [51]:
per_time_inc = 100*(time_opt - baseline_time)/baseline_time
per_time_inc

11.32380694594361

In [52]:
per_co2_dec = 100*(emissions_opt - baseline_emissions)/baseline_emissions
per_co2_dec

-7.468294074048387